# COSC2671 Social Media and Network Analytics

Jeffrey Chan, RMIT University, 2022

In [208]:
import networkx as nx
# python-louvain
import community
import random
import csv


import matplotlib.pyplot as plt
from networkx.algorithms.community import k_clique_communities

In [209]:
# filename of graph to read in
sFilename = "workshop8.graphml"

# filename of ground truth labels
sGroundTruthFilename = "workshop8GroundTruth.csv"

Function to compute the purity of an identified set of communities, 'comm1'.  You'll need to complete the implementation for the workshop.

In [210]:
def purity(comm1, groundTruthComm):
    """
    TODO: Complete implementation

    @param comm1: community to evaluate (list of set format)
    @param groundTruthComm: ground truth to compare against (list of set format)
    @return: purity score, between 0 to 1
    """

    # set overlap
    overlapTotal = 0
    # loop through each community in comm1 (list of sets).  We want to find the overlap with each
    # of the communities in groundTruthComm, take the one with the maximum and add the amount of (maximal)
    # overlap to the running total (overlapTotal)
    #
    for setComm1 in comm1:
        overlaps = []
        for setComm2 in groundTruthComm:
            overlap = setComm2 - setComm1
            overlaps.append(len(overlap))
        overlapTotal += max(overlaps)

    # TODO: remember to normalise the overlapTotal to compute purity
    normalizedOverlap = overlapTotal / comm1.__sizeof__()
    return normalizedOverlap

Utility function to convert the output of python-Louvain, a dictionary (node -> community id), to a list of sets (community id -> [nodes]).

In [211]:
def dictToSetFormat(dComms, maxCommNum):
    """
    Converts dictionary based community output (node -> community id) to list of sets (communities)
    @param dComms: dictionary based community representation
    @param maxCommNum: maximum number of communities
    @return: list of communities representation
    """

    # initialise
    lGroundTruth = [set() for x in range(maxCommNum)]
    # convert each (node : community id) pair to the required set format
    for (name, clusId) in dComms.items():
        lGroundTruth[clusId].add(name)

    return lGroundTruth

Constructs communities from reddit reply graph.

In [212]:
# read the reply graph in
replyGraph = nx.readwrite.read_graphml(sFilename)

#
# compute communities
#

#
# TODO: Implement CPM
#

# k (clique size)
k = 3
# placeholder.  Add the implementation to the right, but maintain the variable name.
lCpmComms = list(k_clique_communities(replyGraph.to_undirected(), k))

#
# TODO: Implement louvain
#

# louvain
# Placeholder, replace right hand side
dLouvainComms = community.best_partition(replyGraph.to_undirected())

# this code converts output of Louvain to the same format as CPM (makes it easier to have one consistent interface
louvanCommNum = max([y for (x,y) in dLouvainComms.items()]) + 1
lLouvainComms = dictToSetFormat(dLouvainComms, louvanCommNum)


#
# TODO: Write out cpm and Louvain values to node attributes of graph
#
# for node in replyGraph.nodes:
#     replyGraph.nodes[node]['CPM'] = -1
#     replyGraph.nodes[node]['Louvain'] = -1

for commId, community in enumerate(lCpmComms):
    for member in community:
        replyGraph.nodes[member]["CPM"] = commId

for commId, community in enumerate(lLouvainComms):
    for member in community:
        replyGraph.nodes[member]["Louvain"] = commId

# for node in replyGraph.nodes:
#     print(replyGraph.nodes[node]['CPM'])
#     print(replyGraph.nodes[node]['Louvain'])


# Output modified graph with prefix mod2 to original file.
nx.readwrite.write_graphml(replyGraph, 'mod2' + sFilename, infer_numeric_types=True)




#
# Compare the communities detected by coparing to a ground truth set of communities
#

lGroundTruth = []
# read in ground truth
with open(sGroundTruthFilename, 'r') as fIn:
    reader = csv.reader(fIn, delimiter=',')

    for lRow in reader:
        lGroundTruth.append(set(lRow))



#
# Compute purity
#

# compute purity
# TODO: implement the purity function, which is defined after this main function
purityValueCpm = purity(lCpmComms, lGroundTruth)
# if using python 2.X, need to replace this print call
print('CPM purity = {}'.format(purityValueCpm))

# convert to set format
purityValueLou = purity(lLouvainComms, lGroundTruth)
print('Louvain purity = {}'.format(purityValueLou))



CPM purity = 5.219827586206897
Louvain purity = 6.4148550724637685
